In [3]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [5]:
load_dotenv()  # Load environment variables from .env file
openai_client = OpenAI()  # Initialize OpenAI client

In [6]:
reader = PdfReader("knowledgebase/Resume-Swapnil Puri_Full_SS.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [7]:
from IPython.display import display, Markdown

In [8]:
display(Markdown("### Extracted Resume Text:"))
display(Markdown(resume[:500] + "..."))  # Display first 500 characters

### Extracted Resume Text:

Resume-Swapnil Puri 
swapnilpuri@hotmail.com 
Ph: 856-313-2878 
 
Educational Qualification: B.Sc. from College of Science, MLS University, 
Udaipur, India (Passed out 1997) 
 
Experience Summary  
Over 17 Years with the following companies:  
1. State Street Corp., USA from Feb 2015 – till date 
2. WIPRO Technologies, USA from July, 2013 – Jan, 2015   
3. Prolifics, US from Feb 2012 – June, 2013 
4. Infosys Technologies, USA from Nov 2010 – Feb 2012  
5. WIPRO Technologies, Chennai from April 2...

In [9]:
with open("knowledgebase/summary.txt", "r") as file:
    summary = file.read()

In [10]:
name = "Swapnil Puri"

In [11]:
system_prompt = f"""
You are an expert career counselor. You have the following resume of a person named {name}
{resume}. You are answering questions based on this resume only. Also, you have the following summary of this person's career objectives and skills:
{summary}. Use this information to answer the questions accurately. The Agent has been instructed to be professional and engaging
"""

In [14]:
def chat_with_resume(message, history):
    messages = [{"role": "system", "content": system_prompt}]+ history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [15]:
gr.ChatInterface(chat_with_resume, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [16]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [17]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of an expert career counselor. You have the following resume of a person named {name} \
{resume}. The Agent has been instructed to be professional and engaging. \
The Agent has been provided with context on {name} in the form of their summary and resume details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [18]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [19]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [20]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [22]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold any certifications?"}]
response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [23]:
display(Markdown("### Reply:"))
display(Markdown(reply))

### Reply:

Yes, I hold several certifications, including:

1. Sun Certified Java Programmer
2. Sun Certified Web Component Developer
3. Sun Certified Business Component Developer
4. Sun Certified Enterprise Architect
5. IBM Certified Solution Developer - Websphere Message Broker V6.0
6. IBM Certified Solution Developer - Websphere Message Broker V7.0
7. IBM Certified Solution Designer - WebSphere MQ V6.0
8. IBM WebSphere DataPower SOA Appliances Firmware V3.8.1, Solution Implementation
9. IBM WebSphere Enterprise Service Bus V7.0, Integration Development
10. TOGAF 9.1 Certification
11. SOA Certified Professional from SOA School

These certifications reflect my commitment to continuous learning and expertise in the technology field.

In [24]:
evaluate(reply, "do you hold any certifications?", messages[:1])

Evaluation(is_acceptable=True, feedback='The agent has provided a perfect response.')

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [27]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [28]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
This answer is unacceptable. The Agent has answered in pig latin, which is unprofessional. The agent should say that based on the resume it is not possible to answer the question.


Traceback (most recent call last):
  File "c:\Swapnil\Code\GenAI\eddoner\LearningLLMCode\.venv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Swapnil\Code\GenAI\eddoner\LearningLLMCode\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Swapnil\Code\GenAI\eddoner\LearningLLMCode\.venv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Swapnil\Code\GenAI\eddoner\LearningLLMCode\.venv\Lib\site-packages\gradio\blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Swapnil\Code\GenAI\eddoner\LearningLLMCode\.venv\Lib\site-packages\gradio\utils.p